## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import numpy as np
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vostok,RU,46.4856,135.8833,3.63,96,60,3.36,broken clouds
1,1,Qaanaaq,GL,77.4840,-69.3632,-13.22,78,100,10.94,overcast clouds
2,2,Punta Arenas,CL,-53.1500,-70.9167,44.62,73,56,21.14,light rain
3,3,Srednekolymsk,RU,67.4500,153.6833,-11.83,99,100,3.29,overcast clouds
4,4,Lagoa,PT,39.0500,-27.9833,58.75,86,100,33.15,overcast clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 80


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
8,8,Georgetown,MY,5.4112,100.3354,76.98,94,40,5.75,thunderstorm with light rain
15,15,Ponta Do Sol,PT,32.6667,-17.1000,67.71,63,2,0.92,clear sky
16,16,Puerto Cabezas,NI,14.0351,-83.3888,78.76,94,75,4.61,drizzle
17,17,Gua Musang,MY,4.8823,101.9644,74.66,98,100,1.23,overcast clouds
21,21,Kununurra,AU,-15.7667,128.7333,78.78,83,100,8.05,overcast clouds
28,28,Atuona,PF,-9.8000,-139.0333,78.55,69,9,15.35,clear sky
34,34,Maamba,ZM,-17.3667,27.1500,69.10,95,100,1.95,overcast clouds
40,40,San Patricio,US,28.0170,-97.5169,73.49,64,10,19.24,clear sky
41,41,Yeppoon,AU,-23.1333,150.7333,71.38,78,18,13.62,few clouds
54,54,Victoria,HK,22.2855,114.1577,77.18,90,36,3.00,scattered clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df[preferred_cities_df.isna().any(axis=1)]
preferred_cities_df.count()

City_ID                169
City                   169
Country                168
Lat                    169
Lng                    169
Max Temp               169
Humidity               169
Cloudiness             169
Wind Speed             169
Current Description    169
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df = preferred_cities_df.dropna()
clean_preferred_cities_df.count()

City_ID                168
City                   168
Country                168
Lat                    168
Lng                    168
Max Temp               168
Humidity               168
Cloudiness             168
Wind Speed             168
Current Description    168
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Georgetown,MY,76.98,thunderstorm with light rain,5.4112,100.3354,
15,Ponta Do Sol,PT,67.71,clear sky,32.6667,-17.1000,
16,Puerto Cabezas,NI,78.76,drizzle,14.0351,-83.3888,
17,Gua Musang,MY,74.66,overcast clouds,4.8823,101.9644,
21,Kununurra,AU,78.78,overcast clouds,-15.7667,128.7333,
28,Atuona,PF,78.55,clear sky,-9.8000,-139.0333,
34,Maamba,ZM,69.10,overcast clouds,-17.3667,27.1500,
40,San Patricio,US,73.49,clear sky,28.0170,-97.5169,
41,Yeppoon,AU,71.38,few clouds,-23.1333,150.7333,
54,Victoria,HK,77.18,scattered clouds,22.2855,114.1577,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

     # 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()    
     
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

hotel_df

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Georgetown,MY,76.98,thunderstorm with light rain,5.4112,100.3354,Cititel Penang
15,Ponta Do Sol,PT,67.71,clear sky,32.6667,-17.1000,Hotel do Campo
16,Puerto Cabezas,NI,78.76,drizzle,14.0351,-83.3888,Cabaña Caribe
17,Gua Musang,MY,74.66,overcast clouds,4.8823,101.9644,Mines Inn Hotel
21,Kununurra,AU,78.78,overcast clouds,-15.7667,128.7333,Hotel Kununurra
...,...,...,...,...,...,...,...
656,Broken Hill,AU,69.51,clear sky,-31.9500,141.4333,Royal Exchange Hotel
659,Hun,LY,79.45,clear sky,29.1268,15.9477,Alrwasi Hotel
661,Beira,MZ,77.09,light rain,-19.8436,34.8389,Beira Terrace Hotel
662,Ures,MX,69.96,clear sky,29.4333,-110.4000,Hacienda Quinta Napoles


In [12]:
# 7. Drop the rows where there is no Hotel Name.
cl_hotel_df = hotel_df.dropna()
cl_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Georgetown,MY,76.98,thunderstorm with light rain,5.4112,100.3354,Cititel Penang
15,Ponta Do Sol,PT,67.71,clear sky,32.6667,-17.1000,Hotel do Campo
16,Puerto Cabezas,NI,78.76,drizzle,14.0351,-83.3888,Cabaña Caribe
17,Gua Musang,MY,74.66,overcast clouds,4.8823,101.9644,Mines Inn Hotel
21,Kununurra,AU,78.78,overcast clouds,-15.7667,128.7333,Hotel Kununurra
...,...,...,...,...,...,...,...
656,Broken Hill,AU,69.51,clear sky,-31.9500,141.4333,Royal Exchange Hotel
659,Hun,LY,79.45,clear sky,29.1268,15.9477,Alrwasi Hotel
661,Beira,MZ,77.09,light rain,-19.8436,34.8389,Beira Terrace Hotel
662,Ures,MX,69.96,clear sky,29.4333,-110.4000,Hacienda Quinta Napoles


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
cl_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in cl_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = cl_hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 
locations = cl_hotel_df[["Lat", "Lng"]]
max_temp = cl_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))